# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('Arrival_time.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Explosive type     588 non-null    object 
 1   Explosive mass     588 non-null    float64
 2   Standoff distance  588 non-null    float64
 3   Time of arrival    588 non-null    float64
dtypes: float64(3), object(1)
memory usage: 18.5+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Explosive type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Explosive mass                588 non-null    float64
 1   Standoff distance             588 non-null    float64
 2   Time of arrival               588 non-null    float64
 3   Explosive type_Composition B  588 non-null    uint8  
 4   Explosive type_TNT            588 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 15.1 KB


In [4]:
dataset.head()

,Explosive mass,Standoff distance,Time of arrival,Explosive type_Composition B,Explosive type_TNT
0,0.5,5.5,0.011771,0,1
1,0.5,6.5,0.014624,0,1
2,0.5,7.5,0.017511,0,1
3,0.5,8.5,0.020422,0,1
4,0.5,9.5,0.023351,0,1


In [5]:
y = dataset[('Time of arrival')]
X = dataset.drop('Time of arrival', axis=1)
print(X.shape, y.shape)

(588, 4) (588,)


In [6]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 71)

In [8]:
X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                y_test,
                                                test_size = 0.5,
                                                random_state = 71)

### Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,0:2] = sc.fit_transform(X_train[:, 0:2])
print (X_train)

In [ ]:
X_test[:,0:2] = sc.transform(X_test[:, 0:2])
print (X_test)

In [ ]:
X_val[:,0:2] = sc.transform(X_val[:, 0:2])
print (X_val)

### S1 -Hyperparameter tuning - neurons, activation function

In [66]:
from tensorflow.keras.layers import Dropout
# Set seed for NumPy
np.random.seed(71)

# Set seed for TensorFlow
tf.random.set_seed(71)

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
def create_model1(last_layer_nodes, activation_func):
    model = Sequential()
    model.add(Dense(units = 200,  input_shape=(X_train.shape[1],), activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(last_layer_nodes, activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear'))
    optimizer1 = Adam(learning_rate=0.01)
    model.compile(optimizer = optimizer1, loss = 'mean_squared_error', metrics = ['mae'])
    return model

activation_func = ['relu', 'leaky_relu', 'softplus']
last_layer_nodes = [10, 20, 30, 40, 50,60, 70, 80,90,100,110,120,130, 140, 150,160, 170, 180, 190, 200]

param_grid = dict( model__activation_func = activation_func,model__last_layer_nodes = last_layer_nodes)

##Wrap model into scikit-learn
model1 = KerasRegressor(model=create_model1, verbose=0, epochs = 100, batch_size = 50, random_state = 71)

In [67]:
kf = KFold(n_splits=5, shuffle=True, random_state=71)
grid1 = GridSearchCV(estimator = model1, param_grid= param_grid, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result1 = grid1.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

D:\Anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.978573 using {'model__activation_func': 'relu', 'model__last_layer_nodes': 100}
-0.005581 (0.003640) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 10}
-0.005461 (0.003942) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 20}
0.566998 (0.467574) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 30}
0.044477 (0.097583) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 40}
0.906980 (0.015514) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 50}
0.936826 (0.008004) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 60}
0.783498 (0.394041) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 70}
0.929319 (0.029428) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 80}
0.928922 (0.035764) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 90}
0.978573 (0.005024) with: {'model__activation_func': 'relu', 'model__last_layer_no

In [68]:
pd.concat([pd.DataFrame(params), pd.DataFrame({'model__first_layer_nodes': [200] * len(params)}),pd.DataFrame(means, columns=['R2'])], axis =1)

,model__activation_func,model__last_layer_nodes,model__first_layer_nodes,R2
0,relu,10,200,-0.005581
1,relu,20,200,-0.005461
2,relu,30,200,0.566998
3,relu,40,200,0.044477
4,relu,50,200,0.906980
5,relu,60,200,0.936826
6,relu,70,200,0.783498
7,relu,80,200,0.929319
8,relu,90,200,0.928922
9,relu,100,200,0.978573


### Merge all files

In [69]:
from pathlib import Path

current_dir = Path.cwd()

#Access input folder
input_dir = Path (current_dir.parent/"Arrival_hyperparameter_tuning_2layers_nodrop_noscale")
print ("1",input_dir)

# Output Excel file
output_excel_file = Path(current_dir.parent/"Arrival_hyperparameter_tuning_2layers_nodrop_noscale/S1_summary.xlsx")

# List to store DataFrames from CSV files
dfs = []

# Loop through CSV files in the directory
for csv_file in input_dir.glob('*.csv'):
    # Read CSV file into a DataFrame and append to the list
    df = pd.read_csv(csv_file)
    dfs.append(df)

# Concatenate DataFrames in the list along rows
merged_df = pd.concat(dfs, ignore_index=True)

# Write the merged DataFrame to an Excel file
merged_df.to_excel(output_excel_file, index=False)

1 G:\Chamodi\Machine_Learning\ANN_1D\Arrival_hyperparameter_tuning_2layers_nodrop_noscale


### S2 - Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [70]:
from tensorflow.keras.optimizers import RMSprop, Nadam
# Set seed for NumPy
np.random.seed(71)

# Set seed for TensorFlow
tf.random.set_seed(71)

# Function to create model, required for KerasClassifier
def create_model2():
 # create model
 model = Sequential()
 model.add(Dense(units=180, input_shape=(X_train.shape[1],), activation='leaky_relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=10, activation='leaky_relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=1, activation='linear'))

 return model

In [71]:
# create model
model2 = KerasRegressor(model=create_model2, verbose=0, random_state = 71, loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [30,40,50]
optimizer = [Adam, Nadam, RMSprop]
learning_rate = [ 0.001,0.01, 0.1]
epochs = [100, 200, 300, 400, 500]

# gridsearch
param_grid2 = dict(batch_size=batch_size, optimizer=optimizer, optimizer__learning_rate = learning_rate, epochs = epochs)
kf = KFold(n_splits=5, shuffle=True, random_state=71)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result2 = grid2.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

D:\Anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.994937 using {'batch_size': 50, 'epochs': 400, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.01}
0.824573 (0.079384) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.001}
0.982589 (0.007157) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.01}
-3.790296 (6.897933) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.1}
-21.959660 (45.650461) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.001}
-0.098562 (1.117883) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.01}
-1.534032 (4.981059) with: {'batch_size': 30, 'epochs': 100, 'optimizer': <class 'keras.src.optimizer

In [72]:
pd.concat([pd.DataFrame(params), pd.DataFrame(means, columns=['R2'])], axis =1)

,batch_size,epochs,optimizer,optimizer__learning_rate,R2
0,30,100,<class 'keras.src.optimizers.adam.Adam'>,0.001,0.824573
1,30,100,<class 'keras.src.optimizers.adam.Adam'>,0.010,0.982589
2,30,100,<class 'keras.src.optimizers.adam.Adam'>,0.100,-3.790296
3,30,100,<class 'keras.src.optimizers.nadam.Nadam'>,0.001,-21.959660
4,30,100,<class 'keras.src.optimizers.nadam.Nadam'>,0.010,-0.098562
...,...,...,...,...,...
130,50,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.010,0.975985
131,50,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.100,-1.033565
132,50,500,<class 'keras.src.optimizers.rmsprop.RMSprop'>,0.001,0.919538
133,50,500,<class 'keras.src.optimizers.rmsprop.RMSprop'>,0.010,0.748353


In [ ]:
# create model
model2 = KerasRegressor(model=create_model2, verbose=0, random_state = 71, loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [30,40,50]
optimizer = [Adam, Nadam, RMSprop]
learning_rate = [ 0.001,0.01, 0.1]
epochs = [100, 200, 300, 400, 500]

# gridsearch
param_grid2 = dict(batch_size=batch_size, optimizer=optimizer, optimizer__learning_rate = learning_rate, epochs = epochs)
kf = KFold(n_splits=5, shuffle=True, random_state=71)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result2 = grid2.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Get the best hyperparameters from the grid search
best_params = grid_result2.best_params_
print (best_params)

In [ ]:
# Create the final model using the best hyperparameters
final_model = KerasRegressor(
    build_fn=create_model2,
    verbose=0,
    random_state=71,
    loss='mean_squared_error',
    metrics=['mae'],
    optimizer = Nadam,
    optimizer__learning_rate = 0.01
    )

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)
# Train the final model on the entire training set
history = final_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                callbacks=[es],
                epochs=best_params['epochs'],
                batch_size=best_params['batch_size'],
                verbose=1)

### Analyse learn history

In [ ]:
# let's see the training and validation accuracy by epoch
history_dict = history.history
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this
epochs = range(1, len(loss_values) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values, 'blue', label='Train set loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation set loss')
#plt.title('Training and testing loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
loss_values1 = history_dict['mae'] # you can change this
val_loss_values1 = history_dict['val_mae'] # you can also change this
epochs = range(1, len(loss_values1) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values1, 'blue', label='Train set MAE')
plt.plot(epochs, val_loss_values1, 'orange', label='Validation set MAE')
#plt.title('Training and testing MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()